# Build Freamework: Cloning and Building Darknet


In [1]:
# clone darknet 
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15150, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 15150 (delta 37), reused 61 (delta 31), pack-reused 15073
Receiving objects: 100% (15150/15150), 13.50 MiB | 8.81 MiB/s, done.
Resolving deltas: 100% (10283/10283), done.


In [2]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [3]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [4]:
# make darknet 
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
  

Using the below helper function to view images

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

# Move files: Uploading Google Drive Files


In [5]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/content
Mounted at /content/gdrive


In [11]:
!ln -s /content/gdrive/Shareddrives/FishOASIS_ML-Detector/ /drive

In [7]:
# cd back into the darknet folder to run detections
%cd ..
%cd /content/darknet

/
/content/darknet


# TRAINING:



In order to create a custom YOLOv4-TINY detector we will need the following:

*   Labeled Dataset
*   .cfg file
*   obj.data and obj.names files
*   train.txt file, test.txt


# Step 1: Gathering and Labeling a Custom Dataset
In order to create an object detector you need a labeled dataset, this may be the most important area to focus on in order to obtain high accuracy models 



# Step 2: Moving Your Custom Dataset Into Your Cloud VM
I recommend renaming the folder with your images and text files on your local machine to be called '**obj**' and then creating a .zip folder of the 'obj' folder. 

In [8]:
!mkdir data/test
!mkdir data/train

In [12]:
# copy the .zip files into the root directory of cloud VM
!cp -r /drive/CLARE/Data/one_train_d data/train
!cp -r /drive/CLARE/Data/one_valid_d data/test


# Step 3: Configuring Files for Training
This step involves properly configuring your custom .cfg file, obj.data, obj.names and train.txt file.


```
set batch=24, this means we will be using 24 images for every training step
set subdivisions=8, the batch will be divided by 8 to decrease GPU VRAM requirement
width= 608 (multiples of 32)
height= 608 (increasing: more prescision, less speed)
maxbatches= max(# classes * 3000, # images, 6000) (How long to train for)
steps=(80% of maxbatch & 90% of maxbatch)
filters=(# of classes + 5) * 3 = 18
for memory issues set random = 0 
```

In [13]:
# upload the yolov4-tiny .cfg back to cloud VM from Google Drive
!cp /drive/CLARE/YOLO_main/yolov4-tiny.cfg ./cfg

Uploading obj.names & obj.data into darknet data folder

In [14]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /drive/CLARE/YOLO_main/obj.names ./data
!cp /drive/CLARE/YOLO_main/obj.data  ./data

Uploading scripts to create a txt file of the train.zip & test.zip

In [15]:
# upload the generate_train.py script to cloud VM from Google Drive
!cp /drive/CLARE/YOLO_main/generate_namelist.py ./

Now run the python script 

In [16]:
!python generate_namelist.py train
!python generate_namelist.py test

In [17]:
# verify train.txt and text.txt can be seen in our darknet/data folder
!ls data/

9k.tree     giraffe.jpg		      labels		scream.jpg  voc.names
coco9k.map  goal.txt		      obj.data		test
coco.names  horses.jpg		      obj.names		test.txt
dog.jpg     imagenet.labels.list      openimages.names	train
eagle.jpg   imagenet.shortnames.list  person.jpg	train.txt


In [18]:
%cd /content/darknet

/content/darknet


# Step 4: Download pre-trained weights for the convolutional layers.
Using pretrained weights allows a more accurate detection with less training time

In [19]:
# upload pretrained convolutional layer weights
# !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights
!cp /drive/CLARE/YOLO_main/yolov4-tiny.conv.29 ./

#Hint:

To avoid colab logging you out hit (CTRL + SHIFT + i) to open consol 

Paste the following code into your console window and hit enter
```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```

# Step 5: Train the model




In [20]:
# cd back into the darknet folder to run detections
%cd ..
%cd /content/darknet

/content
/content/darknet


In [24]:
# train your yolov4-tiny detector
# %%capture training_output
!./darknet detector train data/obj.data cfg/yolov4-tiny.cfg yolov4-tiny.conv.29 -dont_show -map

Streaming output truncated to the last 5000 lines.
 total_bbox = 88670, rewritten_bbox = 0.286455 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 0.000074, iou_loss = 0.000000, total_loss = 0.000074 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.724042), count: 2, class_loss = 0.501398, iou_loss = 6.111750, total_loss = 6.613148 
 total_bbox = 88672, rewritten_bbox = 0.286449 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 0.000048, iou_loss = 0.000000, total_loss = 0.000048 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.456385), count: 1, class_loss = 0.344095, iou_loss = 1.528602, total_loss = 1.872696 
 total_bbox = 88673, rewritten_bbox = 0.286446 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.665992), count: 1, class_loss = 0.020206, iou

In [ ]:
%cd /content/darknet

/content/darknet


In [ ]:
# start training again from previous weights
!./darknet detector train data/obj.data cfg/yolov4-tiny.cfg /mydrive/INTERNS/CLARE/YOLO_main/backup/yolov4-tiny_last.weights -dont_show -map

Start training again from previous weights


# Step 6: Checking the Mean Average Precision (mAP) of Your Model

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-tiny.cfg /mydrive/INTERNS/CLARE/YOLO_main/backup/yolov4-tiny_1000.weights

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    608 x 608 x   3 ->  304 x 304 x  32 0.160 BF
   1 conv     64       3 x 3/ 2    304 x 304 x  32 ->  152 x 152 x  64 0.852 BF
   2 conv     64       3 x 3/ 1    152 x 152 x  64 ->  152 x 152 x  64 1.703 BF
   3 route  2 		                       1/2 ->  152 x 152 x  32 
   4 conv     32       3 x 3/ 1    152 x 152 x  32 ->  152 x 152 x  32 0.426 BF
   5 conv     32       3 x 3/ 1    152 x 152 x  32 ->  152 x 152 x  32 0.426 BF
   6 route  5 4 	                           ->  152 x 152 x  64 
   7 conv     64       1 x 1/ 1    152 x 152 x  64 ->  152 x 152 x  64 0.189 BF
   8 route

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-tiny.cfg /mydrive/INTERNS/CLARE/YOLO_main/backup/yolov4-tiny_2000.weights

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    608 x 608 x   3 ->  304 x 304 x  32 0.160 BF
   1 conv     64       3 x 3/ 2    304 x 304 x  32 ->  152 x 152 x  64 0.852 BF
   2 conv     64       3 x 3/ 1    152 x 152 x  64 ->  152 x 152 x  64 1.703 BF
   3 route  2 		                       1/2 ->  152 x 152 x  32 
   4 conv     32       3 x 3/ 1    152 x 152 x  32 ->  152 x 152 x  32 0.426 BF
   5 conv     32       3 x 3/ 1    152 x 152 x  32 ->  152 x 152 x  32 0.426 BF
   6 route  5 4 	                           ->  152 x 152 x  64 
   7 conv     64       1 x 1/ 1    152 x 152 x  64 ->  152 x 152 x  64 0.189 BF
   8 route

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-tiny.cfg /mydrive/INTERNS/CLARE/YOLO_main/backup/yolov4-tiny_3000.weights

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    608 x 608 x   3 ->  304 x 304 x  32 0.160 BF
   1 conv     64       3 x 3/ 2    304 x 304 x  32 ->  152 x 152 x  64 0.852 BF
   2 conv     64       3 x 3/ 1    152 x 152 x  64 ->  152 x 152 x  64 1.703 BF
   3 route  2 		                       1/2 ->  152 x 152 x  32 
   4 conv     32       3 x 3/ 1    152 x 152 x  32 ->  152 x 152 x  32 0.426 BF
   5 conv     32       3 x 3/ 1    152 x 152 x  32 ->  152 x 152 x  32 0.426 BF
   6 route  5 4 	                           ->  152 x 152 x  64 
   7 conv     64       1 x 1/ 1    152 x 152 x  64 ->  152 x 152 x  64 0.189 BF
   8 route

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-tiny.cfg /mydrive/INTERNS/CLARE/YOLO_main/backup/yolov4-tiny_4000.weights

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    608 x 608 x   3 ->  304 x 304 x  32 0.160 BF
   1 conv     64       3 x 3/ 2    304 x 304 x  32 ->  152 x 152 x  64 0.852 BF
   2 conv     64       3 x 3/ 1    152 x 152 x  64 ->  152 x 152 x  64 1.703 BF
   3 route  2 		                       1/2 ->  152 x 152 x  32 
   4 conv     32       3 x 3/ 1    152 x 152 x  32 ->  152 x 152 x  32 0.426 BF
   5 conv     32       3 x 3/ 1    152 x 152 x  32 ->  152 x 152 x  32 0.426 BF
   6 route  5 4 	                           ->  152 x 152 x  64 
   7 conv     64       1 x 1/ 1    152 x 152 x  64 ->  152 x 152 x  64 0.189 BF
   8 route

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-tiny.cfg /mydrive/INTERNS/CLARE/YOLO_main/backup/yolov4-tiny_5000.weights

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-tiny.cfg /mydrive/INTERNS/CLARE/YOLO_main/backup/yolov4-tiny_final.weights -dont_show < data/train.txt > result.txt

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 2    608 x 608 x   3 ->  304 x 304 x  32 0.160 BF
   1 conv     64       3 x 3/ 2    304 x 304 x  32 ->  152 x 152 x  64 0.852 BF
   2 conv     64       3 x 3/ 1    152 x 152 x  64 ->  152 x 152 x  64 1.703 BF
   3 route  2 		                       1/2 ->  152 x 152 x  32 
   4 conv     32       3 x 3/ 1    152 x 152 x  32 ->  152 x 152 x  32 0.426 BF
   5 conv     32       3 x 3/ 1    152 x 152 x  32 ->  152 x 152 x  32 0.426 BF
   6 route  5 4 	                           ->  152 x 152 x  64 
   7 conv     64       1 x 1/ 1    152 x 152 x  64 ->  152 x 152 x  64 0.189 BF
   8 route  2 7 	                           ->  152 x 152 x 128 
   9 max                2x 2/ 2    152 x 152 x 128 ->   76 x  76 x 128 0.003 BF
  10 con

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-tiny.cfg /mydrive/INTERNS/CLARE/YOLO_main/backup/yolov4-tiny_final.weights -dont_show < data/test.txt > result_test.txt

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 2    608 x 608 x   3 ->  304 x 304 x  32 0.160 BF
   1 conv     64       3 x 3/ 2    304 x 304 x  32 ->  152 x 152 x  64 0.852 BF
   2 conv     64       3 x 3/ 1    152 x 152 x  64 ->  152 x 152 x  64 1.703 BF
   3 route  2 		                       1/2 ->  152 x 152 x  32 
   4 conv     32       3 x 3/ 1    152 x 152 x  32 ->  152 x 152 x  32 0.426 BF
   5 conv     32       3 x 3/ 1    152 x 152 x  32 ->  152 x 152 x  32 0.426 BF
   6 route  5 4 	                           ->  152 x 152 x  64 
   7 conv     64       1 x 1/ 1    152 x 152 x  64 ->  152 x 152 x  64 0.189 BF
   8 route  2 7 	                           ->  152 x 152 x 128 
   9 max                2x 2/ 2    152 x 152 x 128 ->   76 x  76 x 128 0.003 BF
  10 con

# Step 7: Run Your Custom Object Detector!!!
You have done it! You now have a custom object detector to make your very own detections. Time to test it out and have some fun!

In [ ]:
# run your custom detector on image 
!./darknet detector test data/obj.data cfg/yolov4-tiny.cfg /mydrive/YOLO/backup/yolov4-tiny_2700.weights data/test/180716_190328.JPG -Threshold 
imShow('predictions.jpg')

In [ ]:
!ls

sample_data


# Cheers!